In [25]:
import re
import numpy as np
import itertools
import csv

In [2]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrix']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

# Reading in data and running the entire function

In [23]:
filename = r"Xpress Data Files/Data_Xpress_FullDist_Metres.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations')[0])
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrix'))

NbShelves = 96
Shelves = range(1, NbShelves + 1)

FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)


# testing for allocation vector where every shelf is filled
allocations_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                    83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                    90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                    21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                    54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                    1, 33, 59, 42, 52,  45, 79, 39, 68, 73, 53]


# testing where only some of the extra shelves are filled
allocations_mid_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                        83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                        90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                        21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                        54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                        1, 33, 59, 42, 52,  45, 79,  0, 68,  0, 53]


In [26]:
file = open("C:\\Users\\Simeon Horner\\Documents\\GitHub\\RiskAndLogisticsA2\\Data Files\\CSV versions\\OrderList.csv", encoding="utf-8")
csvreader = csv.reader(file)

Orders = []

for row in csvreader:
    Orders.append([int(x) for x in row[1:]])

csvDistances = np.genfromtxt('C:\\Users\\Simeon Horner\\Documents\\GitHub\\RiskAndLogisticsA2\\Data Files\\CSV versions\\DistanceMatrix_meters.csv', delimiter=',', skip_header=0, dtype='i')
csvDistances[0, 0] = 0

In [27]:
csvDistances

array([[ 0,  3,  6, ..., 81, 84,  6],
       [ 3,  0,  3, ..., 84, 81,  9],
       [ 6,  3,  0, ..., 81, 78, 12],
       ...,
       [81, 84, 81, ...,  0,  3, 75],
       [84, 81, 78, ...,  3,  0, 78],
       [ 6,  9, 12, ..., 75, 78,  0]], dtype=int32)

In [5]:
def q1_function(allocation_vector, DistancesMatrix):
    def find_closest_product(current_position, products, distances):
        closest_product = None
        min_distance = float('inf')
        for product in products:
            if distances[current_position][product] <= min_distance:
                closest_product = product
                min_distance = distances[current_position][product]
        return closest_product

    def generate_order_lists(order):
        ''' 
        Takes a list with tuple elements and returns a list of lists 
        with all possible combinations of individual elements individual elements.
        '''
        order_lists = []
        tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
        for combination in itertools.product(*[order[i] for i in tuple_indices]):
            new_order = order.copy()
            for i, index in enumerate(tuple_indices):
                new_order[index] = combination[i]
            order_lists.append(new_order)
        return order_lists

    def greedy_order_route(order, distances):
        ''' 
        Uses a greedy method of calculating the minimum distance. 

        Function has been split into two if statements to consider
        cases of orders where products are contained on more than one shelf.

        If products are contained on more than one shelf, the function constructs 
        a route with all possible shelf combinations and chooses the one with 
        the shortest distance.

        Returns a list containing the route and the total distance for the order.
        '''

        # If all products in the order are contained on one shelf only
        if not any(isinstance(product, tuple) for product in order):
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                    closest_product = find_closest_product(current_position, order, distances)
                    visited.append(closest_product)
                    order.remove(closest_product)
                    current_position = closest_product
            visited.append(0)
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distance_final = OrderDistance
            visited_final = visited

        # If one or more products in the order are contained on more than one shelf
        elif any(isinstance(product, tuple) for product in order):
            order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
            order_routes = []   # initialise a list of routes for all combinations                                 
            order_distances = [] # initialise a list of distances for all combinations
            
            # loop over all combinations
            for order in order_combinations:
                visited = [0]
                current_position = 0  
                for k in range(len(order)):
                    closest_product = find_closest_product(current_position, order, distances)
                    visited.append(closest_product)
                    order.remove(closest_product)
                    current_position = closest_product
                visited.append(0)
                order_routes.append(visited) # add the route for the combination to the list of routes
                OrderDistance = 0
                for i in range(len(visited) - 1):
                    OrderDistance += distances[visited[i]][visited[i+1]]
                order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
            
            # select the order with the shortest distance among the combinations
            min_idx = order_distances.index(min(order_distances))
            visited_final = order_routes[min_idx] 
            order_distance_final = order_distances[min_idx]
                
        return visited_final, order_distance_final # return order route and distance
    
    def convert_orders_to_shelf_indices(allocations):
        ''' 
        This function takes the allocation vector and returns an 
        order matrix with shelf indices instead of product indices.
        '''
        product_to_shelf = {}
        for shelf_index, product in enumerate(allocations):
            if product != 0:  # Check if the element is not zero
                if product not in product_to_shelf:
                    product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
                else:
                    # If the product already exists in the dictionary, append the new shelf index to the list
                    product_to_shelf[product].append(shelf_index + 1)

        # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
        product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

        OrdersByShelf = []
        for order in Orders:
            order_shelf_indices = []
            for product_index in order:
                if product_index in product_to_shelf_tuples:
                    shelf_indices = product_to_shelf_tuples[product_index]
                    order_shelf_indices.append(shelf_indices)
                else:
                    order_shelf_indices.append(0)  # Product not found in allocation matrix
            OrdersByShelf.append(order_shelf_indices)

        return OrdersByShelf
    

    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0           # initialise counter for total distance
    DistancesPerOrder = []      # initalise list to contain the distances for each order 
    routes = []                 # intialise list to contain the routes for each order
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, DistancesMatrix)
        routes.append(visited_order_route)
        DistancesPerOrder.append(visited_order_dist)
        TotalDistance += visited_order_dist

    # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
    SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

    # The indices corresponding to the longest orders (descending order) 
    idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)
    
    return TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, routes

#### Obtaining total distances for different allocations

In [38]:
def getClosestShelves(Distances):
    #Gets the final row of the matrix as this represents the distance to door (packaging)
    distanceToDoor = Distances[0]
    #gets the index, and distance
    indexed_numbers = list(enumerate(distanceToDoor))

    #sorts based on distance and then takes only the index
    sorted_indexed_numbers = sorted(indexed_numbers, key=lambda x: x[1])
    sorted_indexes = [index for index, _ in sorted_indexed_numbers]

    #ignores the first value as this is the door -> door distance = 0
    return sorted_indexes[1:]


getClosestShelves(FullDistanceMatrix)

[1,
 25,
 2,
 26,
 49,
 73,
 3,
 27,
 50,
 74,
 4,
 28,
 51,
 75,
 5,
 29,
 52,
 76,
 6,
 30,
 53,
 77,
 7,
 31,
 54,
 78,
 8,
 32,
 55,
 79,
 9,
 33,
 56,
 80,
 10,
 34,
 57,
 81,
 11,
 35,
 58,
 82,
 12,
 36,
 59,
 83,
 13,
 37,
 60,
 84,
 14,
 38,
 61,
 85,
 15,
 39,
 62,
 86,
 16,
 40,
 63,
 87,
 17,
 41,
 64,
 88,
 18,
 42,
 65,
 89,
 19,
 43,
 66,
 90,
 20,
 44,
 67,
 91,
 21,
 45,
 68,
 92,
 22,
 46,
 69,
 93,
 23,
 47,
 70,
 94,
 24,
 48,
 71,
 95,
 72,
 96]

In [43]:
def addTemporaryFarAwayShelves(Distances):
    #This adds the temporary shelves that are super far away from everything else to the distances matrix
    number_to_fill = 100

    # Number of rows and columns to add
    num_rows_to_add = 90
    num_cols_to_add = 90


    # Create a new row filled with the specified number
    new_row = np.full((1, FullDistanceMatrix.shape[1]), number_to_fill)

    # Repeat the new row to match the number of rows to add
    new_rows = np.repeat(new_row, num_rows_to_add, axis=0)

    # Concatenate the existing array with the new rows
    result_array = np.concatenate((FullDistanceMatrix, new_rows), axis=0)

    new_column = np.full((result_array.shape[0], 1), number_to_fill)
    new_columns = np.repeat(new_column, num_cols_to_add, axis=1)

    result_array = np.concatenate((result_array, new_columns), axis=1)

    return result_array

def getClosestShelves(Distances):
    #Gets the final row of the matrix as this represents the distance to door (packaging)
    distanceToDoor = Distances[0]

    #gets the index, and distance
    indexed_numbers = list(enumerate(distanceToDoor))

    #sorts based on distance and then takes only the index
    sorted_indexed_numbers = sorted(indexed_numbers, key=lambda x: x[1])
    sorted_indexes = [index for index, _ in sorted_indexed_numbers]

    #ignores the first value as this is the door -> door distance = 0
    return sorted_indexes[1:]


def slightlyGreedy_ConstructionHeuristic(Distances):
    refilledProducts = False
    tempProducts = [i for i in range(1, 91)]

    tempDistances = addTemporaryFarAwayShelves(Distances)

    normalShelves = [0]*96
    extraShelves = [x for x in range(1, 91)]

    tempAllocations = normalShelves.copy() + extraShelves.copy()

    orderedShelves = getClosestShelves(Distances)

    print(orderedShelves)

    for o_shelf in orderedShelves:
        shelf = o_shelf-1
        mostReduced = np.inf
        bestP = None
        
        tempShelves = tempAllocations.copy()
        baseline = q1_function(tempShelves, tempDistances)[0]

        for p in tempProducts:
            
            tempShelves = tempAllocations.copy()
            tempShelves[shelf] = p

            newobj = q1_function(tempShelves, tempDistances)

            reduction = newobj[0] - baseline

            if reduction <= mostReduced:
                bestP = p
                mostReduced = reduction

        tempAllocations[shelf] = bestP

        tempProducts.remove(bestP)

        if refilledProducts == False:
            indexOfBestP_onExtraShelf = tempAllocations[96:].index(bestP)
            del tempAllocations[indexOfBestP_onExtraShelf+96]

        if len(tempProducts) == 0:
            tempProducts = [i for i in range(1, 91)]
            refilledProducts = True

        

        print(f"Assigned product {bestP} -> shelf {shelf+1}")

    return tempAllocations

    



In [44]:



results = slightlyGreedy_ConstructionHeuristic(FullDistanceMatrix)

[1, 25, 2, 26, 49, 73, 3, 27, 50, 74, 4, 28, 51, 75, 5, 29, 52, 76, 6, 30, 53, 77, 7, 31, 54, 78, 8, 32, 55, 79, 9, 33, 56, 80, 10, 34, 57, 81, 11, 35, 58, 82, 12, 36, 59, 83, 13, 37, 60, 84, 14, 38, 61, 85, 15, 39, 62, 86, 16, 40, 63, 87, 17, 41, 64, 88, 18, 42, 65, 89, 19, 43, 66, 90, 20, 44, 67, 91, 21, 45, 68, 92, 22, 46, 69, 93, 23, 47, 70, 94, 24, 48, 71, 95, 72, 96]
Assigned product 34 -> shelf 1
Assigned product 31 -> shelf 25
Assigned product 35 -> shelf 2
Assigned product 71 -> shelf 26
Assigned product 70 -> shelf 49
Assigned product 15 -> shelf 73
Assigned product 18 -> shelf 3
Assigned product 30 -> shelf 27
Assigned product 2 -> shelf 50
Assigned product 51 -> shelf 74
Assigned product 76 -> shelf 4
Assigned product 58 -> shelf 28
Assigned product 67 -> shelf 51
Assigned product 6 -> shelf 75
Assigned product 49 -> shelf 5
Assigned product 50 -> shelf 29
Assigned product 11 -> shelf 52
Assigned product 4 -> shelf 76
Assigned product 83 -> shelf 6
Assigned product 28 -> sh

In [45]:
print(results)

[34, 35, 18, 76, 49, 83, 33, 78, 63, 47, 57, 74, 64, 69, 16, 82, 87, 14, 80, 61, 26, 86, 7, 35, 31, 71, 30, 58, 50, 28, 52, 43, 1, 39, 66, 84, 65, 68, 89, 45, 22, 54, 9, 53, 62, 3, 42, 67, 70, 2, 67, 11, 10, 48, 8, 19, 13, 46, 37, 20, 21, 90, 27, 24, 36, 75, 12, 40, 73, 17, 15, 6, 15, 51, 6, 4, 38, 60, 55, 44, 59, 72, 32, 79, 77, 29, 88, 5, 23, 56, 25, 41, 85, 81, 70, 2]


In [48]:
q1_function(results, FullDistanceMatrix)[0]

201888